# This notebook saves HadISST monthly climatologies in a single dataset

#### IMPORTANT: If adapting this code to save other climatologies, be careful to only include full years, as pyLatte will compute monthly climatologies from the saved daily climatologies

In [58]:
import pandas as pd
import xarray as xr
import numpy as np
from pylatte import utils
from ipywidgets import FloatProgress
from dateutil.relativedelta import relativedelta

In [106]:
from myvisualizefuncs import *

#### HadISST dates are referenced to 0000-01-01

In [239]:
path = '/OSM/CBR/OA_DCFP/data/observations/sst/HadISST.nc'

year_shift = 1970 # Year 0000 is a leap year, so is 
da = xr.open_dataset(path, autoclose=True, decode_times=False)['sst']
time_units = 'days since ' + str(year_shift) + '-01-01'

decoded_time = xr.coding.times.decode_cf_datetime(da.time,time_units)
shifted_time = np.array([np.datetime64(time - relativedelta(years=year_shift)).astype('datetime64[ns]') 
                         for time in decoded_time])
da.coords['time'] = ('time', utils.trunc_time(shifted_time,'M'), 
                     {'long_name' : 'time', 'decoded_using' : time_units })

time_use = da['time'][-23:-11].values

/OSM/CBR/OA_DCFP/apps/squ027/anaconda3/envs/pylatte_env/lib/python3.6/site-packages/ipykernel/__main__.py:7: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netcdftime.datetime objects instead, reason: dates out of range


In [240]:
da_clim = da.groupby('time.month').mean(dim='time',keep_attrs=True).compute()
da_clim['month'] = time_use
ds_clim.time.attrs['long_name'] = 'time'
ds_clim.time.attrs['cartesian_axis'] = 'T'
ds_clim.time.attrs['calendar_type'] = 'JULIAN'
ds_clim.time.attrs['bounds'] = 'time_bounds'
da_clim = da_clim.rename({'month' : 'time'})

# Save and delete -----
savename = 'HadISST.clim.nc'

with utils.timer():
    da_clim.to_netcdf(path='/OSM/CBR/OA_DCFP/data/intermediate_products/pylatte_climatologies/' + savename,
                      mode = 'w',
                      encoding = {'time':{'dtype':'float','calendar':'JULIAN',
                                          'units':'days since 0001-01-01 00:00:00'}}) 

   Elapsed: 1.4637062549591064 sec
